In [21]:
import numpy as np
import h5py
import scipy.io
#np.random.seed(1337) # for reproducibility

import keras
# from keras.preprocessing import sequence
# from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
# from keras.regularizers import l2, activity_l1
# from keras.constraints import maxnorm
# from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import EarlyStopping #ModelCheckpoint
# from seya.layers.recurrent import Bidirectional
# from keras.utils.layer_utils import print_layer_shapes
#from keras import backend as K

print ('imports done')

imports done


In [22]:
print ('loading data')
trainmat = h5py.File('everything/train.mat')
validmat = scipy.io.loadmat('everything/valid.mat')
testmat = scipy.io.loadmat('everything/test.mat')
print ('done')

#correcting data format
X_train = np.transpose(np.array(trainmat['trainxdata']),axes=(2,0,1))
y_train = np.array(trainmat['traindata']).T

loading data
done


In [4]:
#look at matrix structure - don't run anymore
print ("test matrix structure:")
for i in testmat: #dictionary with testdata and testxdata
    print (i)
# print (len(testmat["testdata"])) #455024 - N
# print (len(testmat["testdata"][0])) #919 - feature-space, output
# print (len(testmat["testxdata"])) ##455024 - N
# print (len(testmat["testxdata"][0][0])) #4 x 1000
# print (testmat["testxdata"][0]) #one-hot encoding

print ("\n")

print ("train matrix structure:") #transposed
for i in trainmat:
    print (i)
# print (len(trainmat["traindata"])) #919
# print (len(trainmat["traindata"][0])) #4400000 - L
# print (len(trainmat["trainxdata"])) # 1000
# print (len(trainmat["trainxdata"][0][0])) #4 x 4400000
#print (trainmat["trainxdata"][0])#one-hot encoding

print ("\n")

print ("validation matrix structure:")
for i in validmat:
    print (i)
# print (len(validmat["validdata"])) #8000
# print (len(validmat["validdata"][0])) #919 - should be L?
# print (len(validmat["validxdata"])) #8000
# print (len(validmat["validxdata"][0][0])) #4 x 1000

test matrix structure:
__version__
testdata
__header__
testxdata
__globals__


train matrix structure:
traindata
trainxdata


validation matrix structure:
validxdata
validdata
__version__
__header__
__globals__


In [26]:
#building model
model = Sequential()

model.add(Conv1D(input_dim=4,
                        input_length=1000,
                        nb_filter=320,
                        filter_length=26,
                        border_mode="same",
                        activation="relu",
                        subsample_length=1))

model.add(MaxPooling1D(pool_length=13, stride=13)) #wtf why 13

model.add(Dropout(0.2))

# model.add(Conv1D(input_dim=4,
#                         input_length=76,
#                         nb_filter=320,
#                         filter_length=26,
#                         border_mode="same",
#                         activation="relu",
#                         subsample_length=1))

# model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(input_dim=76*640, output_dim=925))
model.add(Activation('relu'))

model.add(Dense(input_dim=925, output_dim=919))
model.add(Activation('sigmoid'))

print 'compiling model'
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='rmsprop', metrics=['accuracy']) #class_mode="binary")

print 'running at most 60 epochs'

#checkpointer = ModelCheckpoint(filepath="DanQ_bestmodel.hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)
callback = keras.callbacks.TensorBoard(log_dir='./logs',
                                       histogram_freq=0,
                                       write_graph=True,
                                       write_grads=False,
                                       write_images=True,
                                       embeddings_freq=0,
                                       embeddings_layer_names=None,
                                       embeddings_metadata=None)

model.fit(X_train[0:10000], y_train[0:10000], 
          batch_size=100, 
          epochs=20,
          shuffle="batch", 
          validation_data=(np.transpose(validmat['validxdata'], axes=(0,2,1)), validmat['validdata']), 
          verbose=1,
          callbacks=[earlystopper, callback])

tresults = model.evaluate(np.transpose(testmat['testxdata'],axes=(0,2,1)), testmat['testdata'])

print tresults

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:10: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", input_shape=(1000, 4), padding="same", strides=1, filters=320, kernel_size=26)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:12: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(strides=13, pool_size=13)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:28: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=925, input_dim=48640)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=919, input_dim=925)`


compiling model
running at most 60 epochs
Train on 10000 samples, validate on 8000 samples
Epoch 1/20
10000/10000 [==============================] - 4s - loss: 148.5882 - acc: 0.0019 - val_loss: 114.6396 - val_acc: 0.0000e+00